# Part 2. Notebook for Network Analysis in Neuroscience

Authors: Eduarda Centeno & Fernando Santos
    
This second part of the project will focus on the 3D visualizations we have developed.


## Table of contents
1. [Imports](#imports)
2. [Short recap - importing data & creating graph object](#short-recap)
3. [Creating 3D visualizations with Plotly](#3d-visualization)

<a id='imports'></a>
###  **1.** Imports

In [ ]:
# Basic data manipulation and visualization libraries
import numpy as np
import matplotlib.pyplot as plt

# Network Libraries
import networkx as nx

# Magic command to load watermark
%load_ext watermark

In [ ]:
# Print versions
%watermark --author "Eduarda & Fernando" --date --time --python --machine --iversion --watermark --packages jupyterlab,notebook

<a id='short-recap'></a>
### **2.** Short recap - importing data & creating graph object

In [ ]:
# Import matrix
matrix = np.genfromtxt('./1000_Functional_Connectomes/Connectivity matrices/AveragedMatrix.txt')

# Absolutize for further user
matrix = abs(matrix)

# Creating Graph
G = nx.from_numpy_matrix(matrix)

# Removing self-loops
G.remove_edges_from(list(nx.selfloop_edges(G)))

# This function accepts a argument 'distance' that, in correlation-based networks, must be seen as the inverse ... 
# ... of the weight value. Thus, a high correlation value (e.g. 0.8) means a shorter distance (i.e 0.2).
G_distance_dict = {(e1, e2): 1 / abs(weight) for e1, e2, weight in G.edges(data='weight')}

# Then add them as attributes to the graph
nx.set_edge_attributes(G, G_distance_dict, 'distance')

# Create graphs from comparison
matrix2 = matrix.copy()
matrix3 = matrix.copy()

# Create sparser graphs
matrix2[matrix2<=0.50] = 0
matrix3[matrix3<=0.75] = 0

st50G = nx.from_numpy_matrix(matrix2)
st25G = nx.from_numpy_matrix(matrix3)

st50G.remove_edges_from(list(nx.selfloop_edges(st50G)))
st25G.remove_edges_from(list(nx.selfloop_edges(st25G)))

<a id='3d-visualization'></a>
### **3.** Creating 3D visualizations with Plotly
First, we need to import the necessary backup script.
Then, it is possible to use a list of values for most of the nodal metrics, e.g. centralities, etc.

In [ ]:
%run "./Background Scripts/HCP_Data_Vis.py"

#### Let's start with the visualization of pairwise interactions

In [ ]:
# Create list of values for a property
values = listnet(nx.closeness_centrality, G, distance='distance')

## Visualization of nodes according to proprieties
# This function will plot the 3D brain with both node size and color according to node_prop
Plot_Brain_Prop(node_prop=values, scale=5) # the second parameter is necessary to scale the node size

In [ ]:
## It is also possible to give a second property, which will change the color of the nodes

# Create a list of a second property
values_2 = listnet(nx.eigenvector_centrality, G, weight='weight')

# Plot 3D brain with node size accoring to values and node colors according to values_2
Plot_Brain_Prop(node_prop=values, scale=5, node_colors=values_2)

In [ ]:
# We have created a degree-specific function for 3D network plotting. 
# Node size will always represent the degree/strength, node color can be changed.

# Plotting with both size and colors according to degree (or strength if weight is True)
degree_3D(st50G, scale=0.2, weight=True)

In [ ]:
# Plotting with size according to degree/strength and colors according another property
degree_3D(st50G, scale=0.2, weight=True, node_colors=values, color_prop_name='Closeness Centrality')

In [ ]:
# Visualization of brain modularity
Plot_Brain_Mod(G, scale=0.75)

In [ ]:
# Visualization of brain network Participation Rank
thr_value = 0.5 # for plot title
clique_size = 2 # this is a line
alpha = 0.1
plotclique3dk(st50G, thr_value, clique_size, alpha) # we will use the st50G to allow a better/clearer visualization

#### Let's now have a look at high-dimensional interactions (TDA)

In [ ]:
# Visualization of triangles (or 3-cliques)
thr_value = 0.5 # for plot title
clique_size = 3 # this is a triangle
alpha = 0.1
plotclique3dk(st50G, thr_value, clique_size, alpha)

In [ ]:
# Visualization of tetrahedrons (or 4-cliques)
thr_value = 0.5 # for plot title
clique_size = 4 # this is a terahedron
alpha = 0.1
plotclique3dk(st50G, thr_value, clique_size, alpha)

In [ ]:
# Visualization of Nodal Curvature
thr_value = 0.5 # for plot title
plotcurv(st50G, thr_value)

##### Acknowledgements

"Data were provided [in part] by the Human Connectome Project, MGH-USC Consortium (Principal Investigators: Bruce R. Rosen, Arthur W. Toga and Van Wedeen; U01MH093765) funded by the NIH Blueprint Initiative for Neuroscience Research grant; the National Institutes of Health grant P41EB015896; and the Instrumentation Grants S10RR023043, 1S10RR023401, 1S10RR019307."